# Example: Regression with `QNNRegressor`

In this example we show how to train the `QNNRegressor` to fit a logarithm. We will also use this demonstrator to show what happens if a QNN, that was trained on a noiseless simulator is executed on a noisy simulator.

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from squlearn import Executor
from squlearn.encoding_circuit import ChebyshevRx
from squlearn.observables import IsingHamiltonian
from squlearn.qnn import QNNRegressor, SquaredLoss
from squlearn.optimizers import SLSQP

We will use the Pennylane quantum framework for executing the circuits.
First we train without any shots.

In [ ]:
executor = Executor("pennylane", shots=None)

## `QNNRegressor` Setup

We start by defining a parameterized quantum circuit (PQC)

In [ ]:
nqubits = 4
number_of_layers = 2

pqc = ChebyshevRx(nqubits, 1, num_layers=number_of_layers)
pqc.draw("mpl")

We also need an observable

In [ ]:
ising_op = IsingHamiltonian(nqubits, I="S", Z="S", ZZ="S")
print(ising_op)

Now we can create the `QNNRegressor` with the PQC and cost operator. We also specify `SquaredLoss` and `SLSQP` as the loss function and optimizer respectively. Finally we specify random initial parameters for both, the PQC and cost operator.

In [ ]:
np.random.seed(13)
param_ini = np.random.rand(pqc.num_parameters)
param_op_ini = np.random.rand(ising_op.num_parameters)

qnn = QNNRegressor(pqc, ising_op, executor, SquaredLoss(), SLSQP(), param_ini, param_op_ini)

## Training

Now we are ready to train the model to fit the logarithm funciton

In [6]:
x_space = np.arange(0.1, 0.9, 0.1)
ref_values = np.log(x_space)

qnn.fit(x_space, ref_values)

test1
value [[ 1.         -0.1051491  -0.5055369 ]
 [ 1.         -0.04909376 -0.34568771]
 [ 1.         -0.04484964 -0.16944648]
 [ 1.         -0.07421503  0.00684104]
 [ 1.         -0.11354335  0.16567007]
 [ 1.         -0.13897573  0.28846595]
 [ 1.         -0.13410135  0.35483913]
 [ 1.         -0.1016551   0.33933274]]
test1
value [[-4.68850329  0.45151084 -0.86243395 -2.82947108 -3.18809407  0.30701888
  -0.58643887 -1.92398712 -2.61033421  2.22643025  0.4216435  -2.82947108
  -1.77497817  1.5139307   0.28670965 -1.92398712]
 [-3.67054952  0.23855176 -0.58838257 -1.99599399 -2.68033196  0.17419678
  -0.42965245 -1.45752739 -2.27513145  2.2120404   1.0379681  -1.99599399
  -1.66136092  1.61529017  0.75795165 -1.45752739]
 [-2.73304187  0.04712012 -0.42062362 -1.34214472 -2.15862407  0.03721664
  -0.33221894 -1.0600591  -1.94422046  1.9421438   1.39354516 -1.34214472
  -1.53559341  1.53395321  1.10065644 -1.0600591 ]
 [-1.9603305  -0.06648693 -0.39296704 -0.89644725 -1.69099043 -0.0

## Evaluation

Let's check the trained parameters of the PQC and operator

In [ ]:
print("Result PQC params:", qnn.param)
print("Result operator params:", qnn.param_op)

Also let's plot the predicted function vs. the actual logarithm function

In [ ]:
x = np.arange(np.min(x_space), np.max(x_space), 0.005)
y = qnn.predict(x)
plt.plot(x, np.log(x))
plt.plot(x, y)

Let's also plot the error of the QNN.

In [ ]:
plt.plot(x, np.abs(y - np.log(x)))

## Evaluation on noisy simulator

Finally let's evaluate the trained `QNNRegressor` with finite sampling noise.

In [ ]:
executor.set_shots(4000)
y = qnn.predict(x)
plt.plot(x, np.log(x))
plt.plot(x, y)

In [ ]:
plt.plot(x, np.abs(y - np.log(x)))

We see, that the model is now subject to noise.